# Import

In [1]:
# import yfinance as yf
import pandas as pd
import os
from os.path import join
from datetime import datetime, timezone, timedelta
import pandas as pd
from yforseer.scraping import update_raw_tables, load_ticker_list, RawDF_Schema
import matplotlib.pyplot as plt
import numpy as np
import logging
data_dir = 'data/yahoo/tickerdaily'

# Scraping

In [ ]:
logging.basicConfig(level=logging.INFO)
ticker_list = load_ticker_list('yforseer/ticker_list.csv')
update_raw_tables(data_dir, ticker_list)

# Combine dataframe

In [9]:
from yforseer.prepare_data import combine_raw_csvs
combine_raw_csvs(raw_data_dir = 'data/yahoo/tickerdaily', 
           save_df_pth = 'data/yahoo/artifacts/combined.csv')



# Select time period, interpolate, stack arrays

In [2]:
from yforseer.prepare_data import extract_features

extract_features('data/yahoo/artifacts/combined.csv',
                 save_npz_pth='data/yahoo/artifacts/Xy.npz',
                 memory=30,
                 lookahead=30)

1COV.DE has 114/4318 = 0.0264 missing values, filled with interpolation.
ALV.DE has 114/4318 = 0.0264 missing values, filled with interpolation.
AMZ.DE has 114/4318 = 0.0264 missing values, filled with interpolation.
APC.DE has 114/4318 = 0.0264 missing values, filled with interpolation.
BAS.DE has 114/4318 = 0.0264 missing values, filled with interpolation.
DHL.DE has 126/4318 = 0.0292 missing values, filled with interpolation.
MBG.DE has 114/4318 = 0.0264 missing values, filled with interpolation.
MSF.DE has 114/4318 = 0.0264 missing values, filled with interpolation.
NVD.DE has 114/4318 = 0.0264 missing values, filled with interpolation.
RHM.DE has 114/4318 = 0.0264 missing values, filled with interpolation.
VOW3.DE has 114/4318 = 0.0264 missing values, filled with interpolation.
X's shape = 
(4258, 11, 30)
y's shape = 
(4258, 11, 4)
